# Throw 40k jobs at a neural network

The actual training was done on GCC
<p>Instance type: `n1-standard-16 (16 vCPUs, 60 GB memory)`

In [ ]:
import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle
import gensim

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import LsiModel

from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel

In [2]:
import spacy
nlp = spacy.load('en')
from spacy.lang.en.stop_words import STOP_WORDS

/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape

/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
/home/goodgame/anaconda3/lib/python3.6/site-packages/msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
/home/goodgame/a

In [3]:
from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

trigram_sentences = LineSentence('spacy_trigram_sentences_PARSED.txt')
word2vec_filepath = 'word2vec_model_all'

In [6]:
%%time

# initiate the model and perform the first epoch of training
skill2vec = Word2Vec(trigram_sentences, size=100, window=5,
                    min_count=20, sg=1, workers=15, iter=15)

skill2vec.save(word2vec_filepath)
print("Done generating model.")

# perform 10 epochs of training
skill2vec.train(trigram_sentences,total_examples=skill2vec.corpus_count, epochs=skill2vec.iter)
skill2vec.save(word2vec_filepath)

In [4]:
# load the finished model from disk
skill2vec = Word2Vec.load(word2vec_filepath)
skill2vec.init_sims()
print(u'{} training epochs so far.'.format(skill2vec.train_count))

2 training epochs so far.


/home/goodgame/anaconda3/lib/python3.6/site-packages/gensim/utils.py:491: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  setattr(self, attrib, None)


In [5]:
print(u'{:,} terms in the food2vec vocabulary.'.format(len(skill2vec.wv.vocab)))


44,673 terms in the food2vec vocabulary.


In [13]:
def get_related_terms(token, topn=20):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in skill2vec.most_similar(positive=[token], topn=topn):

        print(u'{:20} {}'.format(word, round(similarity, 3)))

In [14]:
get_related_terms(u'sql')

SQL                  0.638
PL/SQL               0.634
stored_procedures.   0.623
stored_procedure     0.591
SQLServer            0.585
SQL_statements       0.585
PLSQL                0.576
T-SQL                0.576
DML                  0.572
large_databases      0.571
SQL.                 0.57
query_performance    0.569
query                0.568
OBIEE                0.566
SSIS                 0.566
DB2                  0.563
relational_databases 0.562
TSQL                 0.561
Data_modeling        0.56
relational_databases_such_as 0.559


/home/goodgame/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [25]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = skill2vec.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

In [30]:
word_algebra(add=[u'research', u'big_data'])

data_science


/home/goodgame/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [34]:
word_algebra(add=[u'Hive', u'Postgres'], subtract=[u'MySQL'])

Hadoop


/home/goodgame/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [45]:
word_algebra(subtract=[u'statistics', u'software_engineer'])

Balance


/home/goodgame/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


# Visualize

In [68]:
from sklearn.manifold import TSNE
import pandas as pd

In [72]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in skill2vec.wv.vocab.items()]
print(ordered_vocab[1][2])

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda item: -item[2])

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the skill2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(skill2vec.wv.syn0norm[term_indices, :],
                            index=ordered_terms)

393986


/home/goodgame/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  from ipykernel import kernelapp as app


In [88]:
tsne_input = word_vectors.drop(STOP_WORDS, errors=u'ignore')
tsne_input = tsne_input.head(5000)

In [90]:
tsne_input.head()

0         1         2         3         4         5   \
experience  -0.039959  0.057176 -0.036257 -0.139541 -0.080677 -0.127055   
Experience   0.004817 -0.007835  0.027218 -0.023472 -0.039218 -0.063415   
software    -0.001988  0.051549 -0.089782 -0.048109  0.091702  0.181980   
development  0.018172 -0.020894 -0.195415 -0.006082  0.061313  0.102973   
team         0.129526 -0.002469  0.015829 -0.159259  0.039917  0.066080   

                   6         7         8         9     ...           90  \
experience  -0.006928  0.076205 -0.078392 -0.013559    ...     0.009269   
Experience  -0.133243  0.087012 -0.070254  0.016292    ...     0.036813   
software     0.069399  0.129842 -0.128298  0.041335    ...    -0.084366   
development  0.143018  0.116249 -0.074647  0.072126    ...     0.021295   
team         0.041596  0.157055  0.080325 -0.007167    ...     0.033888   

                   91        92        93        94        95        96  \
experience   0.123594  0.056082  0.094325  0.111970 -0.090186  0.084399   
Experience   0.162738  0.106126  0.108620  0.041196 -0.083480  0.074949   
software     0.118973 -0.024083  0.220807  0.031374 -0.008518  0.117926   
development  0.019664  0.164946  0.108454  0.113397 -0.147010 -0.063146   
team        -0.074242 -0.000360  0.196523  0.071179 -0.065137 -0.176105   

                   97        98        99  
experience  -0.063950  0.050335 -0.153875  
Experience   0.005461  0.100584 -0.040480  
software     0.055087  0.219362  0.079081  
development -0.024878  0.219781 -0.022370  
team        -0.022528  0.190731  0.090708  

[5 rows x 100 columns]

In [91]:
tsne_filepath = u'tsne_model'
tsne_vectors_filepath = u'tsne_vectors.npy'

In [97]:
%%time

tsne = TSNE()
tsne_vectors = tsne.fit_transform(tsne_input.values)

with open(tsne_filepath, 'wb') as f:
    pickle.dump(tsne, f)

pd.np.save(tsne_vectors_filepath, tsne_vectors)

CPU times: user 1min 44s, sys: 9.42 s, total: 1min 54s
Wall time: 1min 54s


In [98]:
with open(tsne_filepath, 'rb') as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=[u'x_coord', u'y_coord'])

In [99]:
tsne_vectors[u'word'] = tsne_vectors.index

In [100]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

output_notebook()

Loading BokehJS ...

In [102]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title=u't-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= (u'pan, wheel_zoom, box_zoom,'
                           u'box_select, reset'),
                   active_scroll=u'wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = u'@word') )

# draw the words as circles on the plot
tsne_plot.circle(u'x_coord', u'y_coord', source=plot_data,
                 color=u'blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color=u'black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value(u'16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);